<a href="https://colab.research.google.com/github/tatsuhiko-suyama/colab_pub/blob/main/3_4_%E5%90%8C%E6%99%82%E5%AD%A6%E7%BF%92.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch==1.12

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 1.9 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.1.0+cu121
    Uninstalling torch-2.1.0+cu121:
      Successfully uninstalled torch-2.1.0+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 1.12.0 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 1.12.0 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 1.12.0 which is incompatible.
torchvision 0.16.0+cu121 requires torch==2.1.0, but you have torch 1.12.0 which is incompatible.


In [7]:
!pip install japanize_matplotlib
!pip install plotly

In [12]:

#@title 関数　ストラングル 3

import japanize_matplotlib
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

import torch

torch.set_default_dtype(torch.float64)

import numpy as np
import time
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
import pandas as pd
#満期の株価
import numpy as np
import pandas as pd
#満期
maturity = 4 # @param

#時間間隔
dt=1/maturity#@param


sigma_ave=0.3 # @param
mu_ave=0.05-sigma_ave**2/2#@param
S_0 = 100  # @param
strike = 120  # @param
branches_bin=2#@param

up_bin =np.exp(mu_ave*dt+sigma_ave*np.sqrt(dt))#@param
# 下落率
down_bin =np.exp(-2*sigma_ave*np.sqrt(dt))#@param
def stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0):
        stock_price=np.array(S_0 * up ** maturity * down ** np.arange(0, (branches - 1) * maturity + 1, 1),dtype=np.float64)
        assert np.min(stock_price)>=0
        return stock_price
#2項モデルでのオプション価格を求め，　それを初期コストとする
risk_neutral_measure=np.array([(1-up_bin*down_bin)/(up_bin-up_bin*down_bin), (up_bin-1)/(up_bin-up_bin*down_bin)])#リスク中立確率を求める
Call_T=np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-strike,0)#満期の支払いを計算
print(Call_T)
for n in np.arange(maturity-1,-1,-1):
  for i in np.arange(0, (branches_bin - 1) * n + 1, 1):

    Call_T[i]=Call_T[i : i+branches_bin]@risk_neutral_measure
init_cost=Call_T[0]



#3項モデルでのパラメータを設定する
branches = 3  # @param
nu=0.005#@param
zeta=0.35#@param
# 上昇率
up = np.exp(nu*dt+zeta*np.sqrt(dt))#@param
# 下落率
down = np.exp(-zeta*np.sqrt(dt))#@param






init_cost
def stock_price_T(maturity=maturity, branches=branches, up=up,down=down, S_0=S_0):
        stock_price=np.array(S_0 * up ** maturity * down ** np.arange(0, (branches - 1) * maturity + 1, 1),dtype=np.float64)
        assert np.min(stock_price)>=0
        return stock_price
#コールオプションの支払い
def call(strike=strike,stock_price=stock_price_T):
    stock_price=stock_price_T()
    return np.maximum(stock_price-strike,0)
#プットオプションの支払い
def put(strike=strike,stock_price=stock_price_T):
    stock_price=stock_price_T()
    return np.maximum(strike-stock_price,0)
#パラメータから確率測度を計算
def param_to_prob(mu, sigma,nu=nu,zeta=zeta,dt=dt):
  prob=np.array([1/2*(sigma/zeta)**2+np.sqrt(dt)/(2*zeta)*(mu-sigma**2/2-nu),1-(sigma/zeta)**2,1/2*(sigma/zeta)**2-np.sqrt(dt)/(2*zeta)*(mu-sigma**2/2-nu)]  )
  assert np.min(prob)>=0#確率測度の定義を満たしているか確認
  assert np.isclose(np.sum(prob),1,atol=1e-15)
  return prob
import numpy as np
def to_numpy(x: torch.Tensor) -> np.ndarray:
    return x.cpu().detach().numpy().reshape(-1)

payoff='call'
#最適ヘッジの計算
def calculate_table(maturity, branches, up, down, S_0,  init_cost,prob_list,strike,payoff):
    #初期条件
    #maturity : 満期
    #branches : 分岐数
    #up : 上昇因子
    #down : 下降因子
    #S_0 : 初期株価
    #init_cost : 初期コスト
    #prob_matrix : 推移確率
    #payoff_func : 支払い関数
    assert (type(maturity)==int) & (maturity>0)#満期は正の整数
    assert (type(branches)==int) & (branches>0)#分岐数は正の整数
    assert up>1 #上昇因子は1より大きい実数
    assert (1>down) & (down>0)#下降因子は0から1までの実数
    assert S_0>=0
    assert init_cost>=0
    import numpy as np
    data = []
    delta_S=[]
    df=pd.DataFrame(columns=['a_n', 'b_n', 'c_n','stock_price', 'Cond_Exp_a', 'Cond_Exp_a_Delta_S', 'Cond_Exp_a_Delta_S_sq', \
                              'Cond_Exp_b', 'Cond_Exp_b_Delta_S','Cond_Exp_c','(Cond_Exp_a_Delta_S)^2/Cond_Exp_a_Delta_S_sq', \
                              '(Cond_Exp_a_Delta_S)(Cond_Exp_b_Delta_S)/Cond_Exp_a_Delta_S_sq','(Cond_Exp_b_Delta_S)^2/Cond_Exp_a_Delta_S_sq'])
    stock_price = torch.from_numpy(stock_price_T()).cuda()
    a_n = torch.from_numpy(np.ones((branches - 1) * maturity + 1)).cuda()

    if payoff=='call':
      #コール
      b_n=torch.clamp(stock_price-strike, min=0).cuda()
      c_n=(torch.clamp(stock_price-strike, min=0).cuda())**2
    if payoff=='strangle':
      #ストラングル
      b_n=torch.clamp(80-stock_price, min=0).cuda()+torch.clamp(stock_price-120, min=0).cuda()
      c_n=(torch.clamp(80-stock_price, min=0).cuda()+torch.clamp(stock_price-120, min=0).cuda())**2
    if payoff=='bull':
      #ブルスプレッド
      b_n=torch.clamp(stock_price-80, min=0).cuda()-torch.clamp(stock_price-120, min=0).cuda()
      c_n=(torch.clamp(stock_price-80, min=0).cuda()-torch.clamp(stock_price-120, min=0).cuda())**2
    elif payoff=='batafrei':
      #バタフライスプレッド
      b_n=torch.clamp(stock_price-80, min=0).cuda()-2*torch.clamp(stock_price-100, min=0).cuda()+torch.clamp(stock_price-120, min=0).cuda()
      c_n=(torch.clamp(stock_price-80, min=0).cuda()-2*torch.clamp(stock_price-100, min=0).cuda()+torch.clamp(stock_price-120, min=0).cuda())**2
    for n in np.arange(maturity-1,-1,-1):
      prob_tensor=prob_list[n]
      a_n= a_n.unfold(0,branches, 1)
      b_n=b_n.unfold(0,branches, 1)
      c_n= c_n.unfold(0,branches, 1)
      stock_price = stock_price.unfold(0,branches,1)
      #条件付き期待値を計算
      Cond_Exp_a=torch.einsum('ij,ij->i', a_n, prob_tensor)
      Cond_Exp_a_Delta_S=torch.einsum('ij,ij->i', a_n*(stock_price-(stock_price[:,0]/up).reshape(-1,1)), prob_tensor)
      Cond_Exp_a_Delta_S_sq=torch.einsum('ij,ij->i', a_n*(stock_price-(stock_price[:,0]/up).reshape(-1,1))**2, prob_tensor)
      Cond_Exp_b=torch.einsum('ij,ij->i', b_n, prob_tensor)
      Cond_Exp_b_Delta_S=torch.einsum('ij,ij->i', b_n*(stock_price-(stock_price[:,0]/up).reshape(-1,1)), prob_tensor)
      Cond_Exp_c=torch.einsum('ij,ij->i', c_n, prob_tensor)

      stock_price=(stock_price/up)[:,0]
      #a, b, cの更新
      a_divide=Cond_Exp_a_Delta_S ** 2 / (Cond_Exp_a_Delta_S_sq)
      a_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
      a_b_divide=(Cond_Exp_a_Delta_S * Cond_Exp_b_Delta_S) / (Cond_Exp_a_Delta_S_sq)
      a_b_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
      b_divide=Cond_Exp_b_Delta_S ** 2 / (Cond_Exp_a_Delta_S_sq)
      b_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
      a_n=Cond_Exp_a-a_divide
      b_n=Cond_Exp_b-a_b_divide
      c_n=Cond_Exp_c-b_divide
    df=pd.concat([df,pd.DataFrame({'a_n' :to_numpy(a_n),'b_n':to_numpy(b_n),'c_n': to_numpy(c_n), 'stock_price': to_numpy(stock_price), 'Cond_Exp_a': to_numpy(Cond_Exp_a), 'Cond_Exp_a_Delta_S': to_numpy(Cond_Exp_a_Delta_S), 'Cond_Exp_a_Delta_S_sq': to_numpy(Cond_Exp_a_Delta_S_sq), 'Cond_Exp_b': to_numpy(Cond_Exp_b), 'Cond_Exp_b_Delta_S': to_numpy(Cond_Exp_b_Delta_S), 'Cond_Exp_c': to_numpy(Cond_Exp_c),
                         '(Cond_Exp_a_Delta_S)^2/Cond_Exp_a_Delta_S_sq' : to_numpy((Cond_Exp_a_Delta_S)**2/Cond_Exp_a_Delta_S_sq),'(Cond_Exp_a_Delta_S)(Cond_Exp_b_Delta_S)/Cond_Exp_a_Delta_S_sq' : to_numpy((Cond_Exp_a_Delta_S)*(Cond_Exp_b_Delta_S)/Cond_Exp_a_Delta_S_sq),'(Cond_Exp_b_Delta_S)^2/Cond_Exp_a_Delta_S_sq' : to_numpy((Cond_Exp_b_Delta_S)**2/Cond_Exp_a_Delta_S_sq)})], ignore_index=True)
    Hedge_Error = a_n*init_cost**2-2*b_n*init_cost+c_n
    pi_0 = Cond_Exp_b_Delta_S/Cond_Exp_a_Delta_S_sq-init_cost*Cond_Exp_a_Delta_S/Cond_Exp_a_Delta_S_sq
    init_cost_opt=df.loc[len(df)-1, 'b_n']/df.loc[len(df)-1, 'a_n']
    Hedge_Error
    return Hedge_Error,pi_0,init_cost_opt,df



#期ごとのNN 2モデル, 多層化 これに決めた　ストラングル 3
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.graph_objects as go
strike_range =[80]

class Normalize(nn.Module):
    def forward(self, x):
        mean = torch.mean(x, dim=0, keepdim=True)
        std = (torch.sqrt(torch.var(x, dim=0, keepdim=True, unbiased=False)+1e-05))
        return (x - mean) / std
normalize = Normalize()
def tensor_to_standard(input_tensor):
  return (input_tensor-torch.mean(input_tensor, dim=0, keepdim=True))/(torch.sqrt(torch.var(input_tensor, dim=0, keepdim=True, unbiased=False)+1e-05))
def decimal_to_base_n(n, base):
    """baseによって任意の進数表示ができる"""
    if n == 0:
        return 0
    nums = []
    while n:
        n, r = divmod(n, base)
        nums.append(str(r))
    return int(''.join(reversed(nums)))
df=pd.DataFrame()
from tqdm import tqdm
for strike in strike_range:

    if payoff=='call':
        #コール
        Call_T=np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-strike,0)#満期の支払いを計算
    if payoff=='strangle':
        #ストラングル
        Call_T=np.maximum(80-stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0),0)+np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)
    if payoff=='bull':
        #ブルスプレッド
        Call_T=np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-80,0)-np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)
    elif payoff=='batafrei':
        #バタフライスプレッド
        Call_T=np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-80,0)-2*np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-100,0)+np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)


    for n in np.arange(maturity-1,-1,-1):
        for i in np.arange(0, (branches_bin - 1) * n + 1, 1):
            Call_T[i]=Call_T[i : i+branches_bin]@risk_neutral_measure
    init_cost=Call_T[0]
    end_epoch = []
    prob_list_bust = []
    df_weight = pd.DataFrame()

    df_pi_0 = pd.DataFrame()
    prob_df = []
    pi_0s=[]
    losses = []
    weights = []
    grads = []
    error_True = []


    def call(strike=strike, stock_price=stock_price_T):
        stock_price = stock_price_T()
        return np.maximum(stock_price - strike, 0)
    #Heの初期化
    def init_weights(m):
        if isinstance(m, nn.Linear):
            nn.init.xavier_normal_(m.weight.data, gain=nn.init.calculate_gain('leaky_relu',0.1))#gain=np.sqrt(2)

    #model_params = np.array([[0.07, 0.35], [0.03, 0.25]])
    model_params = np.array([[0.07, 0.35], [0.03, 0.25],[0.07, 0.25],[0.03, 0.35]])


    models = model_params.shape[0]
    torch.cuda.seed_all()
    torch.manual_seed(0)

    input_size = 9
    hidden_size = 2**5
    output_size = models

    class NN(nn.Module):
        def __init__(self):
            super(NN, self).__init__()
            self.fc_add1=nn.Linear(input_size,input_size)
            self.fc_add2=nn.Linear(input_size,input_size)
            self.layers = nn.Sequential()

            # 入力層
            #1層
            self.layers.add_module("fc_add", nn.Linear(input_size, input_size))


            #2層
            activation=nn.LeakyReLU(0.1)
            self.layers.add_module(f"fc{0}", nn.Linear(input_size,hidden_size))
            self.layers.add_module(f"relu{0}", activation)
            self.layers.add_module(f"st{0}", normalize)
            units=10
            for i in np.arange(1,units,1):
                self.layers.add_module(f"fc{i}", nn.Linear(hidden_size,hidden_size))
                self.layers.add_module(f"relu{i}",activation)
                self.layers.add_module(f"st{i}", normalize)



            #3層


            #4層
            self.layers.add_module(f"fc_final", nn.Linear(hidden_size,hidden_size))
            self.layers.add_module(f"relu_final", activation)


            # 出力層
            self.layers.add_module("output", nn.Linear(hidden_size, output_size))


        def forward(self, x):
            return self.layers(x)


    net =NN().cuda().double()

    net.apply(init_weights)



    optimizer = torch.optim.Adam(net.parameters(), lr=0.00001) #学習率が支配的な感じはある


    num_epochs =10**5
    torch.cuda.synchronize()
    t0 = time.perf_counter()

    for epoch in range(num_epochs):

        optimizer.zero_grad()

        weight_tensor = torch.zeros(maturity, (branches-1) * maturity - 1, model_params.shape[0], requires_grad=True).cuda()
        new_weight_tensor = weight_tensor.clone()


        for n in np.arange(maturity-1, 0, -1):

            S=(torch.from_numpy(stock_price_T(n,branches,up,down,S_0))/S_0).cuda().reshape(-1,1)
            current_time=(torch.ones_like(S)*n*dt).reshape(-1,1)
            input_tensor=torch.concat([current_time,current_time**2,current_time**3,torch.log(S),torch.log(S)**2,torch.log(S)**3,\
                                torch.sqrt(S), torch.sqrt(S)**3 , torch.sqrt(S)**5],dim=1)
            ST_input_tensor=(input_tensor-torch.mean(input_tensor, dim=0, keepdim=True))/(torch.sqrt(torch.var(input_tensor, dim=0, keepdim=True, unbiased=False)+1e-05))

            new_weight_tensor[n][0:(branches-1)*(n+1)-1]=nn.Softmax(dim=1)(net(ST_input_tensor.cuda().double()))

        input_tensor=torch.tensor([[0,0,0,0,0,0,1,1,1],[0,0,0,0,0,0,1,1,1]]).cuda().double()
        ST_input_tensor=(input_tensor-torch.mean(input_tensor, dim=0, keepdim=True))/(torch.sqrt(torch.var(input_tensor, dim=0, keepdim=True, unbiased=False)+1e-05))
        new_weight_tensor[0][0]=nn.Softmax(dim=1)(net(ST_input_tensor.double()))[0]
        weight_tensor = new_weight_tensor

        prob_tensor = torch.zeros(maturity, (branches-1) * maturity - 1, branches, requires_grad=True).cuda().double()
        new_prob_tensor=prob_tensor.clone()
        for m in range(models):
            new_prob_tensor=new_prob_tensor+torch.ones(maturity, (branches-1)*maturity-1, branches, requires_grad=True).cuda()\
                * torch.from_numpy(param_to_prob(model_params[m, 0], model_params[m, 1])).cuda()*new_weight_tensor[:,:,m].reshape(maturity,(branches-1)*maturity-1,1)

        new_prob_tensor
        prob_list=[]
        for n in range(new_prob_tensor.shape[0]):
            prob_list.append(new_prob_tensor[n][0:(branches-1)*n+1].reshape((branches-1)*n+1,branches))

        loss = -calculate_table(maturity, branches, up, down, S_0, init_cost, prob_list,strike,payoff)[0]
        pi_0=calculate_table(maturity, branches, up, down, S_0, init_cost, prob_list,strike,payoff)[1]
        pi_0s.append(pi_0.item())
        weights.append(weight_tensor.detach().cpu().numpy())
        losses.append(-loss.item())
        if epoch % 1000==0:
            print(epoch,-loss.item())
        #print(f'epoch : {epoch},loss : {-loss.item()}')
        loss.backward()




        optimizer.step()

    torch.cuda.synchronize()
    elapsed_time=time.perf_counter() - t0
    print(f'処理時間 : {elapsed_time},strike : {strike},loss : {losses[losses.index(max(losses))]}')

    df=pd.concat([df,pd.DataFrame([strike,losses.index(max(losses)),weights[losses.index(max(losses))][0,0,0],\
                                   pi_0s[losses.index(max(losses))],losses[losses.index(max(losses))],\
                                    int(elapsed_time),np.max(np.gradient(np.maximum.accumulate(losses))[len(losses)-1000:len(losses)])]).T])


    df.to_csv(f'df_Markov_models_{model_params.shape[0]}_maturity_{maturity}_strike_{strike}_hidden_size_{hidden_size}.csv',index=False)
    pd.DataFrame(weights[losses.index(max(losses))].reshape(int(np.size(weight_tensor.cpu().detach().numpy())/models),models))\
        .to_csv(f'weight_strike_{strike}_models_{model_params.shape[0]}_maturity_{maturity}_hidden_size_{hidden_size}_Markov_08.csv',index=False)
    import numpy as np
    import plotly.graph_objects as go

    # 1万個のデータを準備
    losses = np.array(losses)

    # lossesの勾配を計算
    gradient_losses = np.gradient(losses)

    # インデックスの生成（1, 101, 201, ..., 9901）
    indices = np.arange(1, len(losses), 100)

    # lossesとgradient_lossesから指定のインデックスに対応するデータの抽出
    selected_losses = losses[indices]
    selected_gradient_losses = gradient_losses[indices]

    # lossesとgradient_lossesから最大値を取得
    max_loss = np.max(losses)
    max_gradient_loss = np.max(gradient_losses)

    # 最大値のインデックスを取得
    index_max_loss = np.argmax(losses)
    index_max_gradient_loss = np.argmax(gradient_losses)

    # プロットの作成
    fig = go.Figure()

    # lossesの全データと抽出したデータをプロット
    fig.add_trace(go.Scatter(x=np.arange(len(losses)), y=losses, mode='lines', name='Losses'))
    fig.add_trace(go.Scatter(x=indices, y=selected_losses, mode='markers', name='Selected Losses'))
    fig.add_trace(go.Scatter(x=[index_max_loss], y=[max_loss], mode='markers', marker=dict(size=10, color='Red'), name='Max Loss'))

    # gradient_lossesの全データと抽出したデータをプロット
    fig.add_trace(go.Scatter(x=np.arange(len(gradient_losses)), y=gradient_losses, mode='lines', name='Gradient Losses'))
    fig.add_trace(go.Scatter(x=indices, y=selected_gradient_losses, mode='markers', name='Selected Gradient Losses'))
    fig.add_trace(go.Scatter(x=[index_max_gradient_loss], y=[max_gradient_loss], mode='markers', marker=dict(size=10, color='Green'), name='Max Gradient Loss'))

    # グラフのタイトルと軸ラベルの設定
    fig.update_layout(title=f'ストライク {strike}の学習',
                  xaxis_title='エポック数',
                  yaxis_title='ヘッジ誤差')

    # グラフをHTMLファイルとして保存
    fig.write_html(f'hedge_losses_strike_{strike}_models_{models}_maturity_{maturity}_Markov.html')

    # グラフの表示

    step=1
    losses[np.arange(1,len(losses),step)],np.arange(1,len(losses),step)
    pd.DataFrame({'ヘッジ誤差' : losses[np.arange(1,len(losses),step)]},index=np.arange(1,len(losses),step)).\
        to_csv(f'hedge_losses_strike_{strike}_models_{models}_maturity_{maturity}_Markov.csv')
    pd.DataFrame({'ヘッジ誤差' : losses[np.arange(1,len(losses),step)]},index=np.arange(1,len(losses),step))
    losses[np.arange(1,len(losses),step)],np.arange(1,len(losses),step)
    pd.DataFrame({'ヘッジ誤差の差' : np.gradient(losses[np.arange(1,len(losses),step)])},index=np.arange(1,len(losses),step)).\
        to_csv(f'hedge_losses_grad_strike_{strike}_models_{models}_maturity_{maturity}_Markov.csv')
    pd.DataFrame({'ヘッジ誤差の差' : np.gradient(losses)[len(losses)-1000:len(losses)]},index=np.arange(len(losses)-1000,len(losses),1)).\
        to_csv(f'hedge_losses_grad_strike_{strike}_models_{models}_maturity_{maturity}_Markov_final_1000.csv')
    pd.DataFrame({'ヘッジ誤差の差' : np.gradient(losses[np.arange(1,len(losses),step)])},index=np.arange(1,len(losses),step))
    pd.DataFrame(np.maximum.accumulate(losses),index=np.arange(1,len(losses)+step,step)).\
        to_csv(f'hedge_losses_maximum_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov.csv')
    pd.DataFrame(np.gradient(np.maximum.accumulate(losses))[len(losses)-1000:len(losses)],index=np.arange(len(losses)-1000,len(losses),1)).\
        to_csv(f'hedge_losses_maximum_grad_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov_final_1000.csv')
df











[63.12522089 15.6625003   0.          0.          0.        ]
0 1.6587105922816363
1000 4.724999435972336
2000 4.854453917553428
3000 4.925735030523242
4000 4.952421074606832
5000 4.972907827664926
6000 4.991731224112186
7000 5.013359129202286
8000 5.038260851431801
9000 5.074152046155859
10000 5.1158875331886975
11000 5.154397494692603
12000 5.199784757346606
13000 5.24978370518636


KeyboardInterrupt: 

In [28]:

#@title 関数　ストラングル 3

import japanize_matplotlib
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

import torch

torch.set_default_dtype(torch.float64)

import numpy as np
import time
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
import pandas as pd
#満期の株価
import numpy as np
import pandas as pd
#満期
maturity = 4 # @param

#時間間隔
dt=1/maturity#@param


sigma_ave=0.3 # @param
mu_ave=0.05-sigma_ave**2/2#@param
S_0 = 100  # @param
strike = 120  # @param
branches_bin=2#@param

up_bin =np.exp(mu_ave*dt+sigma_ave*np.sqrt(dt))#@param
# 下落率
down_bin =np.exp(-2*sigma_ave*np.sqrt(dt))#@param
def stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0):
        stock_price=np.array(S_0 * up ** maturity * down ** np.arange(0, (branches - 1) * maturity + 1, 1),dtype=np.float64)
        assert np.min(stock_price)>=0
        return stock_price
#2項モデルでのオプション価格を求め，　それを初期コストとする
risk_neutral_measure=np.array([(1-up_bin*down_bin)/(up_bin-up_bin*down_bin), (up_bin-1)/(up_bin-up_bin*down_bin)])#リスク中立確率を求める
Call_T=np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-strike,0)#満期の支払いを計算
print(Call_T)
for n in np.arange(maturity-1,-1,-1):
  for i in np.arange(0, (branches_bin - 1) * n + 1, 1):

    Call_T[i]=Call_T[i : i+branches_bin]@risk_neutral_measure
init_cost=Call_T[0]


dt=1/maturity#@param
#3項モデルでのパラメータを設定する
branches = 3  # @param
nu=0.005#@param
zeta=0.35#@param
# 上昇率
up = np.exp(nu*dt+zeta*np.sqrt(dt))#@param
# 下落率
down = np.exp(-zeta*np.sqrt(dt))#@param






init_cost
def stock_price_T(maturity=maturity, branches=branches, up=up,down=down, S_0=S_0):
        stock_price=np.array(S_0 * up ** maturity * down ** np.arange(0, (branches - 1) * maturity + 1, 1),dtype=np.float64)
        assert np.min(stock_price)>=0
        return stock_price
#コールオプションの支払い
def call(strike=strike,stock_price=stock_price_T):
    stock_price=stock_price_T()
    return np.maximum(stock_price-strike,0)
#プットオプションの支払い
def put(strike=strike,stock_price=stock_price_T):
    stock_price=stock_price_T()
    return np.maximum(strike-stock_price,0)
#パラメータから確率測度を計算
def param_to_prob(mu, sigma,nu=nu,zeta=zeta,dt=dt):
  prob=np.array([1/2*(sigma/zeta)**2+np.sqrt(dt)/(2*zeta)*(mu-sigma**2/2-nu),1-(sigma/zeta)**2,1/2*(sigma/zeta)**2-np.sqrt(dt)/(2*zeta)*(mu-sigma**2/2-nu)]  )
  assert np.min(prob)>=0#確率測度の定義を満たしているか確認
  assert np.isclose(np.sum(prob),1,atol=1e-15)
  return prob
import numpy as np
def to_numpy(x: torch.Tensor) -> np.ndarray:
    return x.cpu().detach().numpy().reshape(-1)

payoff='call'
#最適ヘッジの計算
def calculate_table(maturity, branches, up, down, S_0,  init_cost,prob_list,strike,payoff):
    #初期条件
    #maturity : 満期
    #branches : 分岐数
    #up : 上昇因子
    #down : 下降因子
    #S_0 : 初期株価
    #init_cost : 初期コスト
    #prob_matrix : 推移確率
    #payoff_func : 支払い関数
    assert (type(maturity)==int) & (maturity>0)#満期は正の整数
    assert (type(branches)==int) & (branches>0)#分岐数は正の整数
    assert up>1 #上昇因子は1より大きい実数
    assert (1>down) & (down>0)#下降因子は0から1までの実数
    assert S_0>=0
    assert init_cost>=0
    import numpy as np
    def stock_price_T(maturity=maturity, branches=branches, up=up,down=down, S_0=S_0):
        stock_price=np.array(S_0 * up ** maturity * down ** np.arange(0, (branches - 1) * maturity + 1, 1),dtype=np.float64)
        assert np.min(stock_price)>=0
        return stock_price
    data = []
    delta_S=[]
    df=pd.DataFrame(columns=['a_n', 'b_n', 'c_n','stock_price', 'Cond_Exp_a', 'Cond_Exp_a_Delta_S', 'Cond_Exp_a_Delta_S_sq', \
                              'Cond_Exp_b', 'Cond_Exp_b_Delta_S','Cond_Exp_c','(Cond_Exp_a_Delta_S)^2/Cond_Exp_a_Delta_S_sq', \
                              '(Cond_Exp_a_Delta_S)(Cond_Exp_b_Delta_S)/Cond_Exp_a_Delta_S_sq','(Cond_Exp_b_Delta_S)^2/Cond_Exp_a_Delta_S_sq'])
    stock_price = torch.from_numpy(stock_price_T()).cuda()
    a_n = torch.from_numpy(np.ones((branches - 1) * maturity + 1)).cuda()

    if payoff=='call':
      #コール
      b_n=torch.clamp(stock_price-strike, min=0).cuda()
      c_n=(torch.clamp(stock_price-strike, min=0).cuda())**2
    if payoff=='strangle':
      #ストラングル
      b_n=torch.clamp(80-stock_price, min=0).cuda()+torch.clamp(stock_price-120, min=0).cuda()
      c_n=(torch.clamp(80-stock_price, min=0).cuda()+torch.clamp(stock_price-120, min=0).cuda())**2
    if payoff=='bull':
      #ブルスプレッド
      b_n=torch.clamp(stock_price-80, min=0).cuda()-torch.clamp(stock_price-120, min=0).cuda()
      c_n=(torch.clamp(stock_price-80, min=0).cuda()-torch.clamp(stock_price-120, min=0).cuda())**2
    elif payoff=='batafrei':
      #バタフライスプレッド
      b_n=torch.clamp(stock_price-80, min=0).cuda()-2*torch.clamp(stock_price-100, min=0).cuda()+torch.clamp(stock_price-120, min=0).cuda()
      c_n=(torch.clamp(stock_price-80, min=0).cuda()-2*torch.clamp(stock_price-100, min=0).cuda()+torch.clamp(stock_price-120, min=0).cuda())**2
    for n in np.arange(maturity-1,-1,-1):
      prob_tensor=prob_list[n]
      a_n= a_n.unfold(0,branches, 1)
      b_n=b_n.unfold(0,branches, 1)
      c_n= c_n.unfold(0,branches, 1)
      stock_price = stock_price.unfold(0,branches,1)
      #条件付き期待値を計算
      Cond_Exp_a=torch.einsum('ij,ij->i', a_n, prob_tensor)
      Cond_Exp_a_Delta_S=torch.einsum('ij,ij->i', a_n*(stock_price-(stock_price[:,0]/up).reshape(-1,1)), prob_tensor)
      Cond_Exp_a_Delta_S_sq=torch.einsum('ij,ij->i', a_n*(stock_price-(stock_price[:,0]/up).reshape(-1,1))**2, prob_tensor)
      Cond_Exp_b=torch.einsum('ij,ij->i', b_n, prob_tensor)
      Cond_Exp_b_Delta_S=torch.einsum('ij,ij->i', b_n*(stock_price-(stock_price[:,0]/up).reshape(-1,1)), prob_tensor)
      Cond_Exp_c=torch.einsum('ij,ij->i', c_n, prob_tensor)

      stock_price=(stock_price/up)[:,0]
      #a, b, cの更新
      a_divide=Cond_Exp_a_Delta_S ** 2 / (Cond_Exp_a_Delta_S_sq)
      a_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
      a_b_divide=(Cond_Exp_a_Delta_S * Cond_Exp_b_Delta_S) / (Cond_Exp_a_Delta_S_sq)
      a_b_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
      b_divide=Cond_Exp_b_Delta_S ** 2 / (Cond_Exp_a_Delta_S_sq)
      b_divide[Cond_Exp_a_Delta_S_sq<1e-20]=0
      a_n=Cond_Exp_a-a_divide
      b_n=Cond_Exp_b-a_b_divide
      c_n=Cond_Exp_c-b_divide
    df=pd.concat([df,pd.DataFrame({'a_n' :to_numpy(a_n),'b_n':to_numpy(b_n),'c_n': to_numpy(c_n), 'stock_price': to_numpy(stock_price), 'Cond_Exp_a': to_numpy(Cond_Exp_a), 'Cond_Exp_a_Delta_S': to_numpy(Cond_Exp_a_Delta_S), 'Cond_Exp_a_Delta_S_sq': to_numpy(Cond_Exp_a_Delta_S_sq), 'Cond_Exp_b': to_numpy(Cond_Exp_b), 'Cond_Exp_b_Delta_S': to_numpy(Cond_Exp_b_Delta_S), 'Cond_Exp_c': to_numpy(Cond_Exp_c),
                         '(Cond_Exp_a_Delta_S)^2/Cond_Exp_a_Delta_S_sq' : to_numpy((Cond_Exp_a_Delta_S)**2/Cond_Exp_a_Delta_S_sq),'(Cond_Exp_a_Delta_S)(Cond_Exp_b_Delta_S)/Cond_Exp_a_Delta_S_sq' : to_numpy((Cond_Exp_a_Delta_S)*(Cond_Exp_b_Delta_S)/Cond_Exp_a_Delta_S_sq),'(Cond_Exp_b_Delta_S)^2/Cond_Exp_a_Delta_S_sq' : to_numpy((Cond_Exp_b_Delta_S)**2/Cond_Exp_a_Delta_S_sq)})], ignore_index=True)
    Hedge_Error = a_n*init_cost**2-2*b_n*init_cost+c_n
    pi_0 = Cond_Exp_b_Delta_S/Cond_Exp_a_Delta_S_sq-init_cost*Cond_Exp_a_Delta_S/Cond_Exp_a_Delta_S_sq
    init_cost_opt=df.loc[len(df)-1, 'b_n']/df.loc[len(df)-1, 'a_n']
    Hedge_Error
    return Hedge_Error,pi_0,init_cost_opt,df



#期ごとのNN 2モデル, 多層化 これに決めた　ストラングル 3
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.graph_objects as go
strike_range =[80,90,100,110,120]

class Normalize(nn.Module):
    def forward(self, x):
        mean = torch.mean(x, dim=0, keepdim=True)
        std = (torch.sqrt(torch.var(x, dim=0, keepdim=True, unbiased=False)+1e-05))
        return (x - mean) / std
normalize = Normalize()
def tensor_to_standard(input_tensor):
  return (input_tensor-torch.mean(input_tensor, dim=0, keepdim=True))/(torch.sqrt(torch.var(input_tensor, dim=0, keepdim=True, unbiased=False)+1e-05))
def decimal_to_base_n(n, base):
    """baseによって任意の進数表示ができる"""
    if n == 0:
        return 0
    nums = []
    while n:
        n, r = divmod(n, base)
        nums.append(str(r))
    return int(''.join(reversed(nums)))
df=pd.DataFrame()
from tqdm import tqdm
for strike in strike_range:

    if payoff=='call':
        #コール
        Call_T=np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-strike,0)#満期の支払いを計算
    if payoff=='strangle':
        #ストラングル
        Call_T=np.maximum(80-stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0),0)+np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)
    if payoff=='bull':
        #ブルスプレッド
        Call_T=np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-80,0)-np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)
    elif payoff=='batafrei':
        #バタフライスプレッド
        Call_T=np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-80,0)-2*np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-100,0)+np.maximum(stock_price_T(maturity=maturity, branches=branches_bin, up=up_bin,down=down_bin, S_0=S_0)-120,0)


    for n in np.arange(maturity-1,-1,-1):
        for i in np.arange(0, (branches_bin - 1) * n + 1, 1):
            Call_T[i]=Call_T[i : i+branches_bin]@risk_neutral_measure
    init_cost=Call_T[0]
    end_epoch = []
    prob_list_bust = []
    df_weight = pd.DataFrame()

    df_pi_0 = pd.DataFrame()
    prob_df = []
    pi_0s=[]
    losses = []
    weights = []
    grads = []
    error_True = []


    def call(strike=strike, stock_price=stock_price_T):
        stock_price = stock_price_T()
        return np.maximum(stock_price - strike, 0)
    #Heの初期化
    def init_weights(m):
        if isinstance(m, nn.Linear):
            nn.init.xavier_normal_(m.weight.data, gain=nn.init.calculate_gain('leaky_relu',0.1))#gain=np.sqrt(2)

    #model_params = np.array([[0.07, 0.35], [0.03, 0.25]])
    model_params = np.array([[0.07, 0.35], [0.03, 0.25],[0.07, 0.25],[0.03, 0.35]])


    models = model_params.shape[0]
    torch.cuda.seed_all()
    torch.manual_seed(0)

    input_size = 10
    hidden_size = 2**5
    output_size = models

    class NN(nn.Module):
        def __init__(self):
            super(NN, self).__init__()
            self.fc_add1=nn.Linear(input_size,input_size)
            self.fc_add2=nn.Linear(input_size,input_size)
            self.layers = nn.Sequential()

            # 入力層
            #1層
            self.layers.add_module("fc_add", nn.Linear(input_size, input_size))


            #2層
            activation=nn.LeakyReLU(0.1)
            self.layers.add_module(f"fc{0}", nn.Linear(input_size,hidden_size))
            self.layers.add_module(f"relu{0}", activation)
            self.layers.add_module(f"st{0}", normalize)
            units=10
            for i in np.arange(1,units,1):
                self.layers.add_module(f"fc{i}", nn.Linear(hidden_size,hidden_size))
                self.layers.add_module(f"relu{i}",activation)
                self.layers.add_module(f"st{i}", normalize)



            #3層


            #4層
            self.layers.add_module(f"fc_final", nn.Linear(hidden_size,hidden_size))
            self.layers.add_module(f"relu_final", activation)


            # 出力層
            self.layers.add_module("output", nn.Linear(hidden_size, output_size))


        def forward(self, x):
            return self.layers(x)


    net =NN().cuda().double()

    net.apply(init_weights)



    optimizer = torch.optim.Adam(net.parameters(), lr=0.00001) #学習率が支配的な感じはある


    num_epochs =10**5
    torch.cuda.synchronize()
    t0 = time.perf_counter()

    for epoch in range(num_epochs):

        optimizer.zero_grad()

        weight_tensor = torch.zeros(maturity, (branches-1) * maturity - 1, model_params.shape[0], requires_grad=True).cuda()
        new_weight_tensor = weight_tensor.clone()


        for n in np.arange(maturity-1, 0, -1):

            S=(torch.from_numpy(stock_price_T(n,branches,up,down,S_0))/S_0).cuda().reshape(-1,1)
            current_time=(torch.ones_like(S)*n*dt).reshape(-1,1)
            time_span=(torch.ones_like(S)*dt).reshape(-1,1)
            input_tensor=torch.concat([time_span,current_time,current_time**2,current_time**3,torch.log(S),torch.log(S)**2,torch.log(S)**3,\
                                torch.sqrt(S), torch.sqrt(S)**3 , torch.sqrt(S)**5],dim=1)
            ST_input_tensor=(input_tensor-torch.mean(input_tensor, dim=0, keepdim=True))/(torch.sqrt(torch.var(input_tensor, dim=0, keepdim=True, unbiased=False)+1e-05))

            new_weight_tensor[n][0:(branches-1)*(n+1)-1]=nn.Softmax(dim=1)(net(ST_input_tensor.cuda().double()))

        input_tensor=torch.tensor([[dt,0,0,0,0,0,0,1,1,1],[dt,0,0,0,0,0,0,1,1,1]]).cuda().double()
        ST_input_tensor=(input_tensor-torch.mean(input_tensor, dim=0, keepdim=True))/(torch.sqrt(torch.var(input_tensor, dim=0, keepdim=True, unbiased=False)+1e-05))
        new_weight_tensor[0][0]=nn.Softmax(dim=1)(net(ST_input_tensor.double()))[0]
        weight_tensor = new_weight_tensor

        prob_tensor = torch.zeros(maturity, (branches-1) * maturity - 1, branches, requires_grad=True).cuda().double()
        new_prob_tensor=prob_tensor.clone()
        for m in range(models):
            new_prob_tensor=new_prob_tensor+torch.ones(maturity, (branches-1)*maturity-1, branches, requires_grad=True).cuda()\
                * torch.from_numpy(param_to_prob(model_params[m, 0], model_params[m, 1])).cuda()*new_weight_tensor[:,:,m].reshape(maturity,(branches-1)*maturity-1,1)

        new_prob_tensor
        prob_list=[]
        for n in range(new_prob_tensor.shape[0]):
            prob_list.append(new_prob_tensor[n][0:(branches-1)*n+1].reshape((branches-1)*n+1,branches))

        loss = -calculate_table(maturity, branches, up, down, S_0, init_cost, prob_list,strike,payoff)[0]
        pi_0=calculate_table(maturity, branches, up, down, S_0, init_cost, prob_list,strike,payoff)[1]
        pi_0s.append(pi_0.item())
        weights.append(weight_tensor.detach().cpu().numpy())
        losses.append(-loss.item())
        if epoch % 1000==0:
            print(epoch,-loss.item())
        #print(f'epoch : {epoch},loss : {-loss.item()}')
        loss.backward()




        optimizer.step()

    torch.cuda.synchronize()
    elapsed_time=time.perf_counter() - t0
    print(f'処理時間 : {elapsed_time},strike : {strike},loss : {losses[losses.index(max(losses))]}')

    df=pd.concat([df,pd.DataFrame([strike,losses.index(max(losses)),weights[losses.index(max(losses))][0,0,0],\
                                   pi_0s[losses.index(max(losses))],losses[losses.index(max(losses))],\
                                    int(elapsed_time),np.max(np.gradient(np.maximum.accumulate(losses))[len(losses)-1000:len(losses)])]).T])


    df.to_csv(f'df_Markov_models_{model_params.shape[0]}_maturity_{maturity}_strike_{strike}_hidden_size_{hidden_size}.csv',index=False)
    pd.DataFrame(weights[losses.index(max(losses))].reshape(int(np.size(weight_tensor.cpu().detach().numpy())/models),models))\
        .to_csv(f'weight_strike_{strike}_models_{model_params.shape[0]}_maturity_{maturity}_hidden_size_{hidden_size}_Markov_08.csv',index=False)
    import numpy as np
    import plotly.graph_objects as go

    # 1万個のデータを準備
    losses = np.array(losses)

    # lossesの勾配を計算
    gradient_losses = np.gradient(losses)

    # インデックスの生成（1, 101, 201, ..., 9901）
    indices = np.arange(1, len(losses), 100)

    # lossesとgradient_lossesから指定のインデックスに対応するデータの抽出
    selected_losses = losses[indices]
    selected_gradient_losses = gradient_losses[indices]

    # lossesとgradient_lossesから最大値を取得
    max_loss = np.max(losses)
    max_gradient_loss = np.max(gradient_losses)

    # 最大値のインデックスを取得
    index_max_loss = np.argmax(losses)
    index_max_gradient_loss = np.argmax(gradient_losses)

    # プロットの作成
    fig = go.Figure()

    # lossesの全データと抽出したデータをプロット
    fig.add_trace(go.Scatter(x=np.arange(len(losses)), y=losses, mode='lines', name='Losses'))
    fig.add_trace(go.Scatter(x=indices, y=selected_losses, mode='markers', name='Selected Losses'))
    fig.add_trace(go.Scatter(x=[index_max_loss], y=[max_loss], mode='markers', marker=dict(size=10, color='Red'), name='Max Loss'))

    # gradient_lossesの全データと抽出したデータをプロット
    fig.add_trace(go.Scatter(x=np.arange(len(gradient_losses)), y=gradient_losses, mode='lines', name='Gradient Losses'))
    fig.add_trace(go.Scatter(x=indices, y=selected_gradient_losses, mode='markers', name='Selected Gradient Losses'))
    fig.add_trace(go.Scatter(x=[index_max_gradient_loss], y=[max_gradient_loss], mode='markers', marker=dict(size=10, color='Green'), name='Max Gradient Loss'))

    # グラフのタイトルと軸ラベルの設定
    fig.update_layout(title=f'ストライク {strike}の学習',
                  xaxis_title='エポック数',
                  yaxis_title='ヘッジ誤差')

    # グラフをHTMLファイルとして保存
    fig.write_html(f'hedge_losses_strike_{strike}_models_{models}_maturity_{maturity}_Markov.html')

    # グラフの表示

    step=1
    losses[np.arange(1,len(losses),step)],np.arange(1,len(losses),step)
    pd.DataFrame({'ヘッジ誤差' : losses[np.arange(1,len(losses),step)]},index=np.arange(1,len(losses),step)).\
        to_csv(f'hedge_losses_strike_{strike}_models_{models}_maturity_{maturity}_Markov.csv')
    pd.DataFrame({'ヘッジ誤差' : losses[np.arange(1,len(losses),step)]},index=np.arange(1,len(losses),step))
    losses[np.arange(1,len(losses),step)],np.arange(1,len(losses),step)
    pd.DataFrame({'ヘッジ誤差の差' : np.gradient(losses[np.arange(1,len(losses),step)])},index=np.arange(1,len(losses),step)).\
        to_csv(f'hedge_losses_grad_strike_{strike}_models_{models}_maturity_{maturity}_Markov.csv')
    pd.DataFrame({'ヘッジ誤差の差' : np.gradient(losses)[len(losses)-1000:len(losses)]},index=np.arange(len(losses)-1000,len(losses),1)).\
        to_csv(f'hedge_losses_grad_strike_{strike}_models_{models}_maturity_{maturity}_Markov_final_1000.csv')
    pd.DataFrame({'ヘッジ誤差の差' : np.gradient(losses[np.arange(1,len(losses),step)])},index=np.arange(1,len(losses),step))
    pd.DataFrame(np.maximum.accumulate(losses),index=np.arange(1,len(losses)+step,step)).\
        to_csv(f'hedge_losses_maximum_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov.csv')
    pd.DataFrame(np.gradient(np.maximum.accumulate(losses))[len(losses)-1000:len(losses)],index=np.arange(len(losses)-1000,len(losses),1)).\
        to_csv(f'hedge_losses_maximum_grad_strike_{strike}_models_{models}_maturity_{maturity}_non_Markov_final_1000.csv')
df











[63.12522089 15.6625003   0.          0.          0.        ]
0 2.227315882708126


KeyboardInterrupt: 

In [ ]:

net =NN().cuda().double()
net.apply(init_weights)
loss=torch.zeros(1).cuda()
num_epochs=10**4
losses=[]
for epoch in range(num_epochs):
  optimizer.zero_grad()
  loss=torch.zeros(1).cuda()
  for maturity in [3,4]:
    dt=1/maturity#@param
    #3項モデルでのパラメータを設定する
    branches = 3  # @param
    nu=0.005#@param
    zeta=0.35#@param
    # 上昇率
    up = np.exp(nu*dt+zeta*np.sqrt(dt))#@param
    # 下落率
    down = np.exp(-zeta*np.sqrt(dt))#@param

    weight_tensor = torch.zeros(maturity, (branches-1) * maturity - 1, model_params.shape[0], requires_grad=True).cuda()
    new_weight_tensor = weight_tensor.clone()
    for n in np.arange(maturity-1, 0, -1):
      S=(torch.from_numpy(stock_price_T(n,branches,up,down,S_0))/S_0).cuda().reshape(-1,1)
      current_time=(torch.ones_like(S)*n*dt).reshape(-1,1)
      time_span=(torch.ones_like(S)*dt).reshape(-1,1)
      input_tensor=torch.concat([time_span,current_time,current_time**2,current_time**3,torch.log(S),torch.log(S)**2,torch.log(S)**3,\
                                torch.sqrt(S), torch.sqrt(S)**3 , torch.sqrt(S)**5],dim=1)
      ST_input_tensor=(input_tensor-torch.mean(input_tensor, dim=0, keepdim=True))/(torch.sqrt(torch.var(input_tensor, dim=0, keepdim=True, unbiased=False)+1e-05))

      new_weight_tensor[n][0:(branches-1)*(n+1)-1]=nn.Softmax(dim=1)(net(ST_input_tensor.cuda().double()))

    input_tensor=torch.tensor([[dt,0,0,0,0,0,0,1,1,1],[dt,0,0,0,0,0,0,1,1,1]]).cuda().double()
    ST_input_tensor=(input_tensor-torch.mean(input_tensor, dim=0, keepdim=True))/(torch.sqrt(torch.var(input_tensor, dim=0, keepdim=True, unbiased=False)+1e-05))
    new_weight_tensor[0][0]=nn.Softmax(dim=1)(net(ST_input_tensor.double()))[0]
    weight_tensor = new_weight_tensor

    prob_tensor = torch.zeros(maturity, (branches-1) * maturity - 1, branches, requires_grad=True).cuda().double()
    new_prob_tensor=prob_tensor.clone()
    for m in range(models):
      new_prob_tensor=new_prob_tensor+torch.ones(maturity, (branches-1)*maturity-1, branches, requires_grad=True).cuda()\
                * torch.from_numpy(param_to_prob(model_params[m, 0], model_params[m, 1])).cuda()*new_weight_tensor[:,:,m].reshape(maturity,(branches-1)*maturity-1,1)

    new_prob_tensor
    prob_list=[]
    for n in range(new_prob_tensor.shape[0]):
      prob_list.append(new_prob_tensor[n][0:(branches-1)*n+1].reshape((branches-1)*n+1,branches))

    loss += -calculate_table(maturity, branches, up, down, S_0, init_cost, prob_list,strike,payoff)[0]
  #print(f'ヘッジ誤差 : {loss}')
  losses.append(-loss.item())


  loss.backward()
  optimizer.step()
np.max(losses)


In [ ]:
np.argmax(losses)